__3. Using the supernova.txt data available in https://web.stanford.edu/~hastie/CASI/data.html, predict the magnitude of observations 5,10,15, 25 and 30 fitting a model using all other observations. The candidate models that you must fit are: a) Linear Model using OLS, b) Linear Model using ridge and your choice of Lambda, c) Decision Tree Regressor with your choice of max_depth. Which of these three has the smallest MSE on observations 5,10,15,25 and 30? (30 points). Which are the most relevant variables to predict magnitude according to each model?__


In other to answered the two questions, which model has the smallest MSE on observations 5,10,15,25 and 30? and which are the most relevant variable to predict magnitude according to each model?, we will do the three models and compare them.

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib as cm

from statsmodels.regression import linear_model
from statsmodels.regression.linear_model import OLS #ordinary least squares
from statsmodels.tools.tools import add_constant
from statsmodels.discrete.discrete_model import Poisson

from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

from scipy.special import gammaln
from scipy.optimize import minimize
from scipy import integrate
from sklearn.linear_model import RidgeCV


__Step_1__: Reading the data 

In [2]:
supernova_data = pd.read_table('supernova.txt',sep=r'\s+')
supernova_data.head()

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,Magnitude
SN1,-0.839,-0.927,0.320,0.176,-0.676,-1.272,0.342,-0.427,-0.016,-0.298,-0.543
SN2,-1.892,-0.455,2.407,0.766,-0.944,-1.527,0.088,0.261,0.185,-0.537,2.124
SN3,0.264,-0.803,1.141,-0.863,0.685,-0.354,-1.038,-1.098,-1.319,-1.695,-0.217
SN4,-0.083,1.023,-0.206,-1.115,-0.863,0.715,0.616,0.564,0.615,-0.488,0.946
SN5,0.411,-0.807,-0.129,1.315,-0.647,0.299,-0.822,-1.534,-1.486,-1.087,-3.746


In [3]:
#We separate the 
supernova_train = supernova_data.drop(['SN5','SN10','SN15','SN25','SN30'])
supernova_predict = supernova_data.drop(['SN1','SN2','SN3','SN4','SN6','SN7','SN8',
                                         'SN9','SN11','SN12','SN13','SN14','SN16',
                                         'SN17','SN18','SN19','SN20','SN21','SN22',
                                         'SN23', 'SN24','SN26','SN27','SN28','SN29',
                                        'SN31','SN32','SN33','SN34','SN35','SN36',
                                         'SN37','SN38','SN39'])

# Notice that:
In order to mantain a coherence in the model, we will center and scale respect one of the sets, the supernova_train set and we are going to train our model with that data. Then the supernova_predict set would be scaled and use in the model to see how it works. The scaling and center would help us to see the importance in the coming features, our inputs.

In [4]:
# Since the ten predictor variables are quite varying and not on same scales, we'll need to center and scale them
E1_mean,E1_std=supernova_train['E1'].mean(),supernova_train['E1'].std()
E2_mean,E2_std=supernova_train['E2'].mean(),supernova_train['E2'].std()
E3_mean,E3_std=supernova_train['E3'].mean(),supernova_train['E3'].std()
E4_mean,E4_std=supernova_train['E4'].mean(),supernova_train['E4'].std()
E5_mean,E5_std=supernova_train['E5'].mean(),supernova_train['E5'].std()
E6_mean,E6_std=supernova_train['E6'].mean(),supernova_train['E6'].std()
E7_mean,E7_std=supernova_train['E7'].mean(),supernova_train['E7'].std()
E8_mean,E8_std=supernova_train['E8'].mean(),supernova_train['E8'].std()
E9_mean,E9_std=supernova_train['E9'].mean(),supernova_train['E9'].std()
E10_mean,E10_std=supernova_train['E10'].mean(),supernova_train['E10'].std()

# Center and scale train respect train
supernova_train['E1']=(supernova_train['E1']-E1_mean)/E1_std
supernova_train['E2']=(supernova_train['E2']-E2_mean)/E2_std
supernova_train['E3']=(supernova_train['E3']-E3_mean)/E3_std
supernova_train['E4']=(supernova_train['E4']-E4_mean)/E4_std
supernova_train['E5']=(supernova_train['E5']-E5_mean)/E5_std
supernova_train['E6']=(supernova_train['E6']-E6_mean)/E6_std
supernova_train['E7']=(supernova_train['E7']-E7_mean)/E7_std
supernova_train['E8']=(supernova_train['E8']-E8_mean)/E8_std
supernova_train['E9']=(supernova_train['E9']-E9_mean)/E9_std
supernova_train['E10']=(supernova_train['E10']-E10_mean)/E10_std

# Now we center and scale for the predict set respect to train set
supernova_predict['E1']=(supernova_predict['E1']-E1_mean)/E1_std
supernova_predict['E2']=(supernova_predict['E2']-E2_mean)/E2_std
supernova_predict['E3']=(supernova_predict['E3']-E3_mean)/E3_std
supernova_predict['E4']=(supernova_predict['E4']-E4_mean)/E4_std
supernova_predict['E5']=(supernova_predict['E5']-E5_mean)/E5_std
supernova_predict['E6']=(supernova_predict['E6']-E6_mean)/E6_std
supernova_predict['E7']=(supernova_predict['E7']-E7_mean)/E7_std
supernova_predict['E8']=(supernova_predict['E8']-E8_mean)/E8_std
supernova_predict['E9']=(supernova_predict['E9']-E9_mean)/E9_std
supernova_predict['E10']=(supernova_predict['E10']-E10_mean)/E10_std

In [5]:
#Let's create the columns for the predictions to visualize the predicted values at the end
supernova_predict['Prediction OLS'] = 1
supernova_predict['Prediction Ridge'] = 1
supernova_predict['Prediction Tree'] = 1

In [6]:
#This would be used for the OLS and Ridge
x = add_constant(supernova_predict.drop(['Magnitude','Prediction OLS','Prediction Ridge','Prediction Tree'], axis=1))
x

,const,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10
SN5,1.0,0.481631,-0.794486,-0.183682,1.559922,-0.569026,0.377266,-0.855366,-1.565895,-1.502760,-1.049453
SN10,1.0,-1.673576,-0.780025,-0.146156,-0.517614,0.271401,-0.160014,0.713097,-0.199871,-0.261702,-0.214794
SN15,1.0,1.496996,2.484078,-1.995814,-1.805006,0.405950,2.723178,2.301427,0.820211,0.386363,0.572339
SN25,1.0,0.770707,0.345884,0.322925,3.186758,1.079697,-0.785799,-1.095863,-0.282661,0.691219,1.674325
SN30,1.0,1.218208,0.268414,-0.191582,0.191927,1.959284,0.415791,-0.901374,-0.880912,-0.928451,0.062435


# 1) Linear Regression with OLS


Having reagind the data, we will just need to apply the OLS for our data

In [7]:
x_cols = ['E1','E2','E3','E4','E5','E6','E7','E8','E9','E10']
supernova_OLS = OLS(supernova_train['Magnitude'], add_constant(supernova_train[x_cols])).fit()

In [8]:
#We insert the predicted values in the table
supernova_predict['Prediction OLS'] = supernova_OLS.predict(x)

# 2) Ridge Regression

Here we used cross validation to find the best alpha for our Ridge Regression and have our model

In [9]:
supernova_skR = RidgeCV(alphas=[0.09,0.1,0.11,0.12,0.13,0.14,0.15]).fit(add_constant(supernova_train[x_cols]),supernova_train['Magnitude'])

In [10]:
model_cv = supernova_skR.fit(add_constant(supernova_train[x_cols]),supernova_train['Magnitude'])
alp=model_cv.alpha_
print('The best alpha with cross validation is',alp)

The best alpha with cross validation is 0.15


In [11]:
supernova_ridge = linear_model.OLS(supernova_train['Magnitude'], add_constant(supernova_train[x_cols])).fit_regularized(alpha=alp/supernova_train.shape[0], L1_wt=0)
#We insert the predicted values in the table
supernova_predict['Prediction Ridge'] = supernova_ridge.predict(x)
supernova_ridge.params #Here we have the coefficients for the parameters


array([ 0.14623719, -0.52533127,  0.57661627, -0.10647332, -0.0411362 ,
        0.03182501, -0.36276359,  0.74191783, -0.21686775,  0.53432215,
       -0.00714745])

# 3) Decision Tree Regressor


In the following, we are going to try the DecisionTreeRegressor for several values of max_depth. We would choose the one that reduce the MSE for the values to predict  5,10,15,25,30.

In [12]:
MSEold=10000
x_tree = ['Magnitude','Prediction OLS','Prediction Ridge','Prediction Tree']
for i in range(2,30):
    for j in range(100):
        reg_rtree = DecisionTreeRegressor(max_depth=i).fit(supernova_train.drop('Magnitude',1), supernova_train['Magnitude'])
        x_pred = reg_rtree.predict(supernova_predict.drop(x_tree,1))
        MSEnew = mean_squared_error(supernova_predict['Magnitude'],x_pred)
        if (MSEnew<MSEold):
            MSEold = MSEnew
            bestreg_rtree=reg_rtree
            #We insert the predicted values in the table
            supernova_predict['Prediction Tree'] = x_pred
            max_depth=i
            

In [13]:
print('max_depth =',max_depth)
print('MSE=',MSEold)
#Here we 

max_depth = 8
MSE= 0.6280886000000001


Our max_depth for this case is 8

# Errors and importance features

Now, we have our three models, the OLS, the ridge with alpha=0.15 and the decission tree regressor with max_depth=7. Now we can se the predicted values for wach model in the next table

In [14]:
#Let's see the values for each model
supernova_predict

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,Magnitude,Prediction OLS,Prediction Ridge,Prediction Tree
SN5,0.481631,-0.794486,-0.183682,1.559922,-0.569026,0.377266,-0.855366,-1.565895,-1.502760,-1.049453,-3.746,-1.847282,-1.854948,-2.279
SN10,-1.673576,-0.780025,-0.146156,-0.517614,0.271401,-0.160014,0.713097,-0.199871,-0.261702,-0.214794,0.576,1.130292,1.113290,0.903
SN15,1.496996,2.484078,-1.995814,-1.805006,0.405950,2.723178,2.301427,0.820211,0.386363,0.572339,3.103,1.862804,1.835924,3.075
SN25,0.770707,0.345884,0.322925,3.186758,1.079697,-0.785799,-1.095863,-0.282661,0.691219,1.674325,-2.833,-0.268837,-0.299624,-3.754
SN30,1.218208,0.268414,-0.191582,0.191927,1.959284,0.415791,-0.901374,-0.880912,-0.928451,0.062435,-2.092,-1.395568,-1.389173,-2.272


Let's see the MSE for the predicted values of 5,10,15,25 and 30 in each model.

In [15]:
MSE_OLS=mean_squared_error(supernova_predict['Magnitude'],supernova_predict['Prediction OLS'])
MSE_RID=mean_squared_error(supernova_predict['Magnitude'],supernova_predict['Prediction Ridge'])
MSE_Tree=mean_squared_error(supernova_predict['Magnitude'],supernova_predict['Prediction Tree'])
print('MSE_OLS =',MSE_OLS)
print('MSE_RID =',MSE_RID)
print('MSE_Tree =',MSE_Tree)

MSE_OLS = 2.502080910743422
MSE_RID = 2.4764398828020155
MSE_Tree = 0.6280886000000001


# First question-Notice that:
The smallest MSE is for the decission tree regressor model.

Then, we can see the most important features in each model 

In [16]:
#For the OLS model, here we can look for the coefficients.
supernova_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Magnitude   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     10.42
Date:                Thu, 30 Jul 2020   Prob (F-statistic):           2.19e-06
Time:                        14:47:23   Log-Likelihood:                -39.950
No. Observations:                  34   AIC:                             101.9
Df Residuals:                      23   BIC:                             118.7
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1469      0.163      0.899      0.378      -0.191       0.485
E1            -0.5250      0.364     -1.444      0.162      -1.277       0.227
E2             0.5764      0.485      1.188      0.247      -0.427       1.580
E3            -0.0973      0.516     -0.188      0.852      -1.166       0.971
E4            -0.0339      0.466     -0.073      0.943      -0.997       0.929
E5             0.0383      0.328      0.117      0.908      -0.640       0.717
E6            -0.3558      0.621     -0.573      0.572      -1.641       0.929
E7             0.7712      0.702      1.099      0.283      -0.681       2.223
E8            -0.2718      0.644     -0.422      0.677      -1.603       1.060
E9             0.5819      0.833      0.699      0.492      -1.140       2.304
E10           -0.0155      0.504     -0.031      0.976      -1.058       1.028
==============================================================================
Omnibus:                        2.276   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.320   Jarque-Bera (JB):                1.251
Skew:                           0.049   Prob(JB):                        0.535
Kurtosis:                       2.065   Cond. No.                         17.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
#For the Ridge model, here we can look for the coefficients.
supernova_ridge.params
#Here the first value is the coefficient for the constant, so we have to look in the other components

array([ 0.14623719, -0.52533127,  0.57661627, -0.10647332, -0.0411362 ,
        0.03182501, -0.36276359,  0.74191783, -0.21686775,  0.53432215,
       -0.00714745])

In [18]:
#For the decission tree regressor model, here we can look for the coefficients.
feat_imps = bestreg_rtree.feature_importances_
feat_imp_df = pd.DataFrame({'variable_name': supernova_train.drop('Magnitude',1).columns , 'importance': feat_imps}) \
                .sort_values('importance', ascending=False)
feat_imp_df.head(10)

,variable_name,importance
7,E8,0.571369
6,E7,0.218551
3,E4,0.066892
2,E3,0.040843
1,E2,0.040049
8,E9,0.031287
5,E6,0.014144
0,E1,0.013006
9,E10,0.002548
4,E5,0.001312


# Second question-We need to notice that:
For each model the most 4 important features  are:

- OLS model -> E7,E9,E2,E1
- Ridge model-> E7,E2,E9,E1
- Tree model -> E8,E7,E4,E3

in their respective order, from the most to the less important

__Coefficients in OLS:__
- E7->0.7712
- E9->0.5819
- E2->0.5764
- E1->-0.5250

__Coefficients in Ridge:__
- E7->0.74191783
- E2-> 0.57661627
- E9-> 0.53432215
- E1->-0.52533127

__Importance in Tree:__
- E8->0.572898
- E7->0.217316
- E4->0.067275
- E3->0.045333

# ---------------------------------------------------------------------------------------------

# Try with the most important components

Let's try to improve our models just taking the __two most important components__

In [19]:
#Knowing the previous information, we take just the first two most important components for each model
#For OLS we use E7 and E9
#For Ridge E7 and E2
#For Tree E8 and E7
NewtryOLS_train = supernova_train.drop(['E1','E2','E3','E4','E5','E6','E8','E10'],1)
NewtryOLS_predict = supernova_predict.drop(['E1','E2','E3','E4','E5','E6','E8','E10','Prediction OLS','Prediction Ridge','Prediction Tree'],1)
NewtryRidge_train = supernova_train.drop(['E1','E3','E4','E5','E6','E8','E9','E10'],1)
NewtryRidge_predict = supernova_predict.drop(['E1','E3','E4','E5','E6','E8','E9','E10','Prediction OLS','Prediction Ridge','Prediction Tree'],1)
NewtryTree_train = supernova_train.drop(['E1','E2','E3','E4','E5','E6','E9','E10'],1)
NewtryTree_predict = supernova_predict.drop(['E1','E2','E3','E4','E5','E6','E9','E10','Prediction OLS','Prediction Ridge','Prediction Tree'],1)

In [20]:
#Our new model for OLSE
x = add_constant(NewtryOLS_predict.drop(['Magnitude'],axis=1))
x_cols = ['E7','E9']
supernova_ols = OLS(NewtryOLS_train['Magnitude'], add_constant(NewtryOLS_train[x_cols])).fit()

In [21]:
#Our new model for Ridge
x_cols = ['E2','E7']
supernova_skR = RidgeCV(alphas=[0.09,0.1,0.11,0.12,0.13,0.14,0.15]).fit(add_constant(NewtryRidge_train[x_cols]),NewtryRidge_train['Magnitude'])
model_cv = supernova_skR.fit(add_constant(NewtryRidge_train[x_cols]),NewtryRidge_train['Magnitude'])
alp=model_cv.alpha_
supernova_ridge = linear_model.OLS(NewtryRidge_train['Magnitude'], 
                                   add_constant(NewtryRidge_train[x_cols])).fit_regularized(alpha=alp/NewtryRidge_train.shape[0], L1_wt=0)

In [22]:
#Our new model for Decission Tree Regressor
MSEold=10000
x_tree = ['Magnitude']
for i in range(2,30):
    for j in range(100):
        reg_rtree = DecisionTreeRegressor(max_depth=i).fit(NewtryTree_train.drop('Magnitude',1), NewtryTree_train['Magnitude'])
        x_pred = reg_rtree.predict(NewtryTree_predict.drop('Magnitude',1))
        MSEnew = mean_squared_error(NewtryTree_predict['Magnitude'],x_pred)
        if (MSEnew<MSEold):
            MSEold = MSEnew
            bestreg_rtree_new=reg_rtree
            max_depth_new=i

In [23]:
print('max_depth =',max_depth_new)
print('MSE=',MSEold)

max_depth = 9
MSE= 0.18595240000000007


In [24]:
MSE_OLS_new=mean_squared_error(NewtryOLS_predict['Magnitude'], supernova_ols.predict(x))
MSE_RID_new=mean_squared_error(NewtryRidge_predict['Magnitude'],supernova_ridge.predict(x))
MSE_Tree_new=mean_squared_error(NewtryTree_predict['Magnitude'],bestreg_rtree_new.predict(NewtryTree_predict.drop('Magnitude',1)))
print('MSE_OLS_new =',MSE_OLS_new)
print('MSE_RID_new =',MSE_RID_new)
print('MSE_Tree_new =',MSE_Tree_new)

MSE_OLS_new = 1.7979628868739794
MSE_RID_new = 5.096587786339689
MSE_Tree_new = 0.18595240000000007


# Now we can see that:
There was and improvement for the OLS and Tree models. Nevertheless, the ridge model goes worse than the ridge model with all the parameters. 